[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/farahbakhsh3/X1/blob/master/Ferdosi2.ipynb)

In [6]:
# !wget https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt

--2018-10-16 21:05:07--  https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/laax0sxqzzlddtq/ferdosi.txt [following]
--2018-10-16 21:05:07--  https://www.dropbox.com/s/raw/laax0sxqzzlddtq/ferdosi.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc70ce578ad10b17e1d78d6911b4.dl.dropboxusercontent.com/cd/0/inline/ATKDy1HFyZShMWQBr9lk9PiuuBafBem5mjlpP1nGe09KWO2mvNOV8Ger5QPlP0NOCVNh0WiL8jbsVARRWVx8TjJy0MHMlJ5HDMWboWmRI9JARed59RQybYSO3jMEZdAj4rxLlk17eRS4ezBskwCSbAYw8GNR4965tK0Z3AbJOhV-viyiHDW2SvsGwukdPebbn7M/file [following]
--2018-10-16 21:05:07--  https://uc70ce578ad10b17e1d78d6911b4.dl.dropboxusercontent.com/cd/0/inline/ATKDy1HFyZShMWQBr9lk9PiuuBafBem5mjlpP1nGe09KWO2mvNO

In [0]:
import random
 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LSTM, GRU, CuDNNGRU, CuDNNLSTM 
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences


In [8]:
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    texts = f.read()
print('corpus length:', len(texts))

texts = texts.replace('\n', ' ** ')
texts = texts.split(' ')

corpus length: 2545108


In [9]:
max_length = 10
step = 1
sentences = []
next_words = []
for i in range(0, len(texts) - max_length, step):
    sentences.append(texts[i: i + max_length])
    next_words.append(texts[i + max_length])
print('nb sequences:', len(sentences))

nb sequences: 660966


In [10]:
print(sentences[0])
print(next_words[0])

['به', 'نام', 'خداوند', 'جان', 'و', 'خرد', '**', 'کزین', 'برتر', 'اندیشه']
برنگذرد


In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19257 unique tokens.


In [0]:
len_of_words = len(word_index)
embeding_dim = 100

word2idx = dict((c, i) for i, c in enumerate(word_index))
idx2word = dict((i, c) for i, c in enumerate(word_index))

In [13]:
x = pad_sequences(sequences, maxlen=max_length)
# x = np.array(sequences)
y = np.zeros((len(sequences)), dtype=int)
for i, sentence in enumerate(sentences):
    try:
        y[i] = int(word2idx[next_words[i]])
    except:
        pass

print('Shape of data tensor:', x.shape)
print('Shape of label tensor:', y.shape)

Shape of data tensor: (660966, 10)
Shape of label tensor: (660966,)


In [14]:
print(1, [idx2word[idx-1] for idx in x[0]])
print(2, idx2word[y[0]])

print(3, x[0])
print(5, word2idx[next_words[0]])
print(6, idx2word[word2idx[next_words[0]]])

1 ['به', 'نام', 'خداوند', 'جان', 'و', 'خرد', '**', 'کزین', 'برتر', 'اندیشه']
2 برنگذرد
3 [  3  84 356 116   2 102   1 359 686 232]
5 3630
6 برنگذرد


In [0]:
def on_epoch_end(epoch, _):
    if epoch % 5 != 0:
        return
    
    start_index = random.randint(0, len(texts) - max_length - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = ' '.join(texts[start_index: start_index + max_length])
        print(sentence)
        generated = generated.join(sentence)
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(400):
            x_pred = np.zeros((1, max_length), dtype=int)
            for t, word in enumerate(sentence.split(' ')):
        #         print(t, word)
                x_pred[0, t] = word2idx[word]

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_word = idx2word[pred_id]

            generated = generated + ' ' + next_word
            sentence = ' '.join(sentence.split(' ')[1:])
            sentence = sentence + ' ' + next_word

        print(generated.replace(' ** ', '\n'))

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
# model = models.Sequential()
# model.add(layers.Embedding(len(word_index), 100))
# model.add(layers.LSTM(128))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


model = Sequential()
model.add(Embedding(len_of_words, embeding_dim))
# model.add(CuDNNGRU(256))
model.add(GRU(256))
model.add(Dense(len_of_words, activation='softmax'))
 
model.compile(optimizer=RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
print(model.summary())
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         1925700   
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_3 (Dense)              (None, 19257)             4949049   
Total params: 7,148,925
Trainable params: 7,148,925
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/60
220032/660966 [========>.....................] - ETA: 2:07 - loss: 5.8899 - acc: 0.2180